## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-03-20-52-37 +0000,nypost,"Trump pushes for NJ, VA upsets despite not wan...",https://nypost.com/2025/11/03/us-news/trump-pu...
1,2025-11-03-20-48-26 +0000,nyt,"With Acquisition, Kimberly-Clark Bets That Tyl...",https://www.nytimes.com/2025/11/03/science/tyl...
2,2025-11-03-20-47-00 +0000,wsj,U.S. Natural Gas Extends Gains on Weather Outlook,https://www.wsj.com/articles/european-gas-rise...
3,2025-11-03-20-45-42 +0000,nyt,"After Hurricane Melissa, a Seaside Town in Jam...",https://www.nytimes.com/2025/11/03/world/ameri...
4,2025-11-03-20-45-00 +0000,wsj,Treasury Yields Lose Steam Despite Hawkish Fed...,https://www.wsj.com/articles/u-s-treasury-like...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2319/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,40
236,will,13
196,new,13
241,china,9
252,mamdani,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
134,2025-11-03-15-34-50 +0000,bbc,Trump says it would be 'hard for me' to fund N...,https://www.bbc.com/news/articles/c231e284345o...,90
210,2025-11-03-09-49-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...,87
256,2025-11-03-02-20-39 +0000,nypost,"Trump talks NYC mayoral race, government shutd...",https://nypost.com/2025/11/02/us-news/trump-ta...,85
40,2025-11-03-19-19-01 +0000,nyt,Will Trump’s Tariff Deal Tilt the Playing Fiel...,https://www.nytimes.com/2025/11/02/business/ec...,78
29,2025-11-03-19-43-42 +0000,startribune,Trump administration will provide partial SNAP...,https://www.startribune.com/trump-administrati...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
134,90,2025-11-03-15-34-50 +0000,bbc,Trump says it would be 'hard for me' to fund N...,https://www.bbc.com/news/articles/c231e284345o...
132,48,2025-11-03-15-44-45 +0000,nypost,Kimberly-Clark buys Tylenol maker Kenvue in a ...,https://nypost.com/2025/11/03/business/tylenol...
140,48,2025-11-03-15-23-21 +0000,nypost,Chilling video shows UK mass train stabbing su...,https://nypost.com/2025/11/03/world-news/video...
210,47,2025-11-03-09-49-00 +0000,wsj,Trump Says Xi Will Help Fight Fentanyl. Will C...,https://www.wsj.com/world/trump-says-xi-will-h...
151,47,2025-11-03-14-32-42 +0000,nypost,Antisemitic activist Linda Sarsour vows to hol...,https://nypost.com/2025/11/03/us-news/linda-sa...
42,42,2025-11-03-19-16-00 +0000,wsj,Federal Reserve governor Lisa Cook said she su...,https://www.wsj.com/economy/central-banking/em...
20,40,2025-11-03-20-07-00 +0000,wsj,The U.S. said it would use emergency funds to ...,https://www.wsj.com/politics/policy/u-s-to-pay...
143,39,2025-11-03-15-09-25 +0000,bbc,Israeli military's ex-top lawyer arrested over...,https://www.bbc.com/news/articles/cy0kpd97qqko...
145,37,2025-11-03-15-02-27 +0000,nypost,Michelle Obama moans she endured ‘white hot gl...,https://nypost.com/2025/11/03/us-news/michelle...
105,36,2025-11-03-17-01-00 +0000,wsj,"Hundreds of travel businesses, including casin...",https://www.wsj.com/politics/policy/travel-ind...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
